## Lithosphere Dynamics

### Introduction

Computational lithospheric dynamics is a vast topic, and this notebook serves as an introduction, not a comprehensive review. To learn more, please review the additional examples in ASPECT (cookbooks, benchmarks), the ASPECT source code, and the extensive literature on the subject, including other lithospheric dynamic codes. Non-linear lithospheric dynamics is difficult so it is a good idea to properly set your expectations for the amount of time it takes to conduct a novel study.

In this tutorial, we will go over modeling the relationships between crustal deformation, lithospheric deformation, and mantle convection. Remember, these processes occur over time and spatial scares which can vary dramatically:

![timescales](timescales.png) 


### Numerical Modeling

Here is an overview of the steps involved in working numerical modeling problems:

1. Define what physical processes to examine
2. Capture the physics through equations
3. Choose a numerical approximation to solve the equations
4. Design, write, and validate software (or validate existing code)
5. Formulate a hypothesis to test
6. Design and run a series of models
7. Verify and interpret results

Do not skip steps 1 and 5 under any circumstances.

### Capture Physics Through Equations

In this case, the governing equations describe incompressible viscous flow:



1. Stokes Equations

    $\nabla * u = 0 $
    
    $\nabla * \sigma' -\nabla P +\rho g = 0$ 
    
    In case you are unfamiliar with these equations, let's take a moment to understand what they mean.
    
    In the first of the Stokes equations, $\nabla *$ denotes divergence, while $u$ denotes the vector field of the fluid's velocity. The divergence of a vector field at a certain point is a measure of the change of the field's density at that point. A divergence of zero means that the field's density is constant at that point. The first equation states that for any point within the vector field of the fluid's velocity, the divergence remains the same. In other words, despite the fact that the fluid is flowing, the fluid remains at a constant density throughout. This is the same thing as saying that the fluid is incompressible, which makes sense because these equations describe incompressible viscous flow. This first equation is the mass conservation equation.
    
    In the second of the stokes equations, $\sigma'$ denotes the stress tensor, $\nabla$ denotes the gradient, (note: $\nabla$ differs from $\nabla*$ as they are both differential operators, but different kinds) $P$ denotes pressure, $\rho$ denotes density, and $g$ denotes gravity. The gradient of a vector field is a function describing the direction and rate in which the vector field increases the most for each point in the field. When a point within the vector field is taken as input into the gradient operator, the return value is a vector whose direction signifies the direction at that point in the flow field in which the value of the field increases the most. The magnitude of this vector signifies the rate of greatest increase at that point. 
    
    The second equation represents a balance of Pressure, Viscous, and Gravity forces. It is also the applciation of the second law of motion to an incompressible fluid, ignoring effect of inertia on the fluids acceleration (it is reasonable to ignore inertia because of the viscocity and time scale of mantle convection). 

    The term $-\nabla P +\rho g$ represents the pressure and gravity forces, while the term $\nabla * \sigma' $ represents the viscoelastic force.
    

2. Conservation of Energy

    $\rho c (\frac{\delta T}{\delta t}+u*\nabla T) = \nabla * K\nabla T +H $
    
    This equation describes how energy is conserved in this model of incompressible viscous flow. Here, $c$ denotes heat capacity at constant temperature, $K$ denotes thermal conductivity, $\frac{\delta T}{\delta t}$ denotes the derivative of temprature T with respect to time t, and H denotes the heat radiated by the fluid. 
    
    The equation states that the only increase in temperature is due to internal heat production ($+H$). In compressible fluids, temperature and density are related and so more terms are needed on the right hand side of the equation to account for temperature changes caused by compression. However, since we are modelling incompressible viscous flow, density is constant and so the above equation is accurate. It describes the evolution of $T$ temperature as a scalar field (meaning, the temperature of the fluid across it's entire area/volume) over time.


3. Rheology

    Rheology includes nonlinear viscous flow like diffusion, dislocation and composite creep, viscoelasticity, and plasticity. Each of these relationships are described by nonlinear systems of equations which are not trivial to solve. For complex simulations, you may need to heavily limit the number of nonlinear iterations per time step in order to run a model in a reasonable amount of time.
    
### Formulate a Hypothesis to Test

When investigating lithosphere dynamics, a few areas invite study through geodynamic modeling, such as continental extension, subduction, and thrust sheets. In these cases rheology, rates of deformation, and the initial lithospheric structure (among other attributes) control the resulting patterns of deformation. When designing a hypothesis to test, it is important to think about which of these controls can and should be modelled. 

### Tutorial: Viscoelastic Bending Beam

Viscoelasticity is especially important to describing lithosphere dynamics in regions experiencing flexure. An example of viscoelastic behavior might be an oceanic plate undergoing subduction, or a plate straining under the weight of material erupted by volcanism.

ASPECT includes viscoelastic and viscoplastic material models and an elastic rheology model for simulating viscoelastic deformation. A number of benchmark .prm files make use of these material and rheology models, and are found in the /benchmarks directory within an ASPECT installation. These benchmarks serve as tools to verify that ASPECT solves various problems accurately. Alone, they are not necesarily intended as accurate models of geophysics processes. Rather, the benchmarks represent simplified scenarios in which to test ASPECT's adherance to physical laws. 

In this tutorial, we will be using the viscoelastic_bending_beam.prm file. This benchmark consists of a dense viscoelastic beam within a low density viscous fluid. Both materials are subjected to the force of gravity. Over the course of the simulation, gravity causes the dense beam to flex downward within the viscous fluid. The beam doesn't sink because one of its sides is "connected" to the side of the 2D box by designating that side as a "no-slip" surface.

In order to verify the elasticity of the beam, we will stop applying the force of gravity partway through the simulation. If everything is working correctly, we should see the beam rebound to it's original position after the force of gravity is eliminated from the model. 

There are a few steps to follow in order to cancel gravity partway through the simulation and allow the beam to rebound. 
Note that in the # Global parameters of the .prm file, we set Resume computation to auto:
 

With the *resume computation* parameter set to *auto*, we can run ASPECT using the same .prm file and the model will automatically continue at the time step where it last stopped, rather than restarting at the first time step. This feature enables us to change model parameters after a run, and then continue the simulation with new parameters. After the simulation finishes, let's turn off gravity by changing the magnitude from 10 to 0:

During its last run, the model reached the end time specified in the .prm file and stopped running. Accordingly, we need to increase the end time before running the model again. If we don't, the model will resume running at the "end" time step and immediately stop because it has already reached the last specified time.

Let's increase the run time to 250 Kyr:

If we run the model again it will resume from where we left off. You should now see the viscoelastic beam rebound.

Viewing the viscoelastic stresses in the model over time is also insightful. It should reveal that viscoelastic stresses build and relax on the beam as it bends. With greater downward flexure, the top of the beam experiences greater tension while the bottom experiences compression. When gravity is removed from the simulation, these stresses both return to zero as the beam rebounds to its original position.

### Tutorial: Continental Extension

Divergent plate boundary forces give rise to magmatism as well as varied deformation mechanisms including brittle, viscous, and elastic deformation. These are processes controlling lithosphere dynamics in continental rift zones, which can be investigated using ASPECT models.

The continental extension tutorial uses a 2D box model, which represents a vertical cross section through an extension zone. Divergent forces work on the the model, applying tension outward on the box horizontally. At the same time, upwelling mantle material applies an upward force on the bottom of the box. Continental crust makes up the top 30km of the box, while the mantle forms the bottom 70km. Their respective densities are 2700$kg/m^3$ and 3300$kg/m^3$ which correspond to the real world values. Temperature increases from 500°C at the crust/mantle boundary to 1200°C at the bottom boundary of the box.

The top portion of the model is a [free surface](https://aspect-documentation.readthedocs.io/en/latest/user/methods/freesurface/index.html), where the mesh can be deformed by forces from within the model. This is meant to represent the surface of the lithosphere, above which there is insufficient material to prevent the flow of rocks.) where the mesh can be deformed by forces from within the model. This is meant to represent the surface of the lithosphere, above which there is insufficient material to prevent the flow of rocks. 

After the simulation is complete, we will notice that the free surface along the top of the 2D box has developed topography similar to what we see in real-world rift valleys.

Now, let's look at this tutorial's .prm file in detail.
#### Model Geometry and Pre-refinement Grid Spacing

in the Geometry model subsection, the Geometry model is defined as a box subdivided into sub-boxes which are each 20 kilometers square. 

Then, the model is further subdivided. Look at the comment above the Mesh refinement subsection: "Adaptively refine the mesh to 2.5 km spacing above y=50 km and between x=40 and x=160 km." 

The Mesh refinement subsection sets the initial global refinement to 2, meaning that at the start of the simulation all of the 20km square boxes are refined twice. In a single refinement, each box is split into four boxes of equal size to each other (in three dimensions, each cube is split into 8 cubes per refinement). With an initial global refinement of 2, each box refined twice. The first refinement splits each 20km box into four 10km square boxes, and the second refinement produces four 5km square boxes from each of those.

The minimum refinement function subsection is what adaptively refines the mesh to 2.5km above y=50km and between x=40 and x=160 km. Notice that the coordinates are specified in the conditional section of the if statement(the first parameter in the parenthesis), while the values 3 and 2 are specied in the then(the second parameter) and the else(the third parameter) section like this:

set Function expression = if (**conditional**,**then**,**else**)

Refer to the manual [here](https://aspect-documentation.readthedocs.io/en/stable/user/run-aspect/parameters-overview/muparser-format.html#sec-run-aspect-parameters-overview-muparser-format) for more information on how to write function expressions in .prm files.

This function expression will set the refinement of all the blocks within the area defined in the conditional statement three times, and will refine all other blocks two times.


# NOTE FOR CIG EDUCATION: This .prm file is not the same as the continental extension cookbook, they have modified it a bit for the tutorial. The tutorial is found in the 2020 tectonics modeling tutorial on Rene's github.


#### Solver settings

Only two parameters in this subsection have been changed from their default values, the Nonlinear Newton solver switch tolerance parameter and the Use Eisenstat walker for Picard iterations parameter.

The Nonlinear Newton solver switch tolerance parameter has a default value of 1e-5, which is less tolerant than the value we have chosen. This means that ASPECT will take longer to switch from using Picard iterations to using Newton iterations. More information about nonlinear Newton and Picard solvers can be found in in the [ASPECT manual](https://aspect-documentation.readthedocs.io/en/latest/user/methods/nonlinear-stokes-solvers.html). In short, the Picard solvers will almost almost converge on a solution but require more iterations whereas Newton solvers can converge very quickly on a solution once they are close to it, but are at risk of failing to find a solution at all. However, Newton solvers can be slower than Picard solvers even if they use fewer iterations, so choosing the proper tolerance perameter is important whenever you are designing a new ASPECT model.

The Eisenstat walker method is used to determine how accurately linear systems need to be solved, and by default is only used in the Newton iterations. By setting this parameter to true, we are also using the Eisenstat Walker method in the Picard iterations, which means the program might not ever need to switch the Newton solver on.

More information about solver parameters is available in the aspect manual [here](https://aspect-documentation.readthedocs.io/en/stable/parameters/Solver_20parameters.html).

### Velocity Boundary Conditions

This part of the .prm file use a function expression to designate the boundary forces in this model, similar to the expression used for geometry model refinement.

### Initial Composition

Note that the "Names of fields" parameter does not mean define rock types or chemical compositions, but rather fields which are useful in setting up this model. Compositional fields have many uses in ASPECT, which you can read about [here](https://aspect-documentation.readthedocs.io/en/stable/user/methods/compositional-fields.html#sec-methods-compositional-fields).

### Temperature Boundary Conditions & Heating Model

The most important part of this subsection is that by leaving out the left and right sides from "set Fixed temperature boundary indicators," those sides are defined to be insulating.

Here, make sure that the compositional heating values are consistent with the initial temperature model, which is also defined in the .prm file.

### Changes to try on your own

If you want to try more, here are some parameters that you can try changing on your own:
- Time step size
- Model Geometry
- Initial Lithology
- Initial Temperature
- Initial Plastic Strain
- Solver Convergence Settings
- Viscous Flow Law
- Brittle Yield Mechanism
- Brittle parameters